In [1]:
!pip install -q tokenizers
!pip install -q transformers
!pip install -q tokenizers

In [2]:
import numpy as np
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


In [3]:
# True if no flanks
hairpin_bool = False
if hairpin_bool:
  relevant_seq = 'pre_mirna'
else:
  relevant_seq = 'full_seq'

In [ ]:
# all data:
# gff_data = pd.read_csv('/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/Data_output/gff_output/sebastian_db_features_new.csv')
# mirogen_data = pd.read_csv('/sise/vaksler-group/IsanaRNA/Transformers/Rom/Data_output/miRGeneDB_output/miRGeneDB_features.csv')

# train data:
gff_data = pd.read_csv('/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/seq_clusters/gff_train_data.csv')
mirogen_data = pd.read_csv('/sise/vaksler-group/IsanaRNA/Transformers/GPT_env/seq_clusters/mirgene_train_data.csv.csv')

# # remove the flanks from the sequences:
# gff_data = gff_data.dropna(subset=['full_seq', 'flank1', 'flank2']); mirogen_data = mirogen_data.dropna(subset=['full_seq', 'flank1', 'flank2']) # found some null flanks
# for index, row in gff_data.iterrows():
#     gff_data.at[index, 'full_seq'] = gff_data.at[index, 'full_seq'].replace(row['flank1'], '').replace(row['flank2'], '')
# for index, row in mirogen_data.iterrows():
#     mirogen_data.at[index, 'full_seq'] = mirogen_data.at[index, 'full_seq'].replace(row['flank1'], '').replace(row['flank2'], '')

original_gff_sequences = gff_data[relevant_seq].tolist() ; original_mirogen_sequences  = mirogen_data[relevant_seq].tolist()
gff_data = gff_data.dropna(subset=['Star','Mature']); mirogen_data = mirogen_data.dropna(subset=['Star','Mature']) # found some null star and mature


def encode_sequence(row):
    full_seq = row[relevant_seq]
    mature_start = full_seq.find(row['Mature'])
    mature_end = mature_start + len(row['Mature'])
    star_start = full_seq.find(row['Star'])
    star_end = star_start + len(row['Star'])
    # star_end = len(row['Star'])

    encoded_seq = full_seq
    if star_start < star_end:
        if mature_start < star_start:
            encoded_seq = (encoded_seq[:mature_start] + 'ZZZZZ' +
                           encoded_seq[mature_start:mature_end] + 'BBBBB' +
                           encoded_seq[mature_end:star_start] + 'DDDDD' +
                           encoded_seq[star_start:star_end] + 'FFFFF' +
                           encoded_seq[star_end:])
        else:
            encoded_seq = (encoded_seq[:star_start] + 'DDDDD' +
                           encoded_seq[star_start:star_end] + 'FFFFF' +
                           encoded_seq[star_end:mature_start] + 'ZZZZZ' +
                           encoded_seq[mature_start:mature_end] + 'BBBBB' +
                           encoded_seq[mature_end:])
    return encoded_seq


def decode_sequence(encoded_seq):
    decoded_seq = encoded_seq.replace('ZZZZZ', '').replace('BBBBB', '').replace('DDDDD', '').replace('FFFFF', '')
    return decoded_seq


# Apply the encoding function on both datasets
gff_data['encoded_seq'] = gff_data.apply(encode_sequence, axis=1) ; mirogen_data['encoded_seq'] = mirogen_data.apply(encode_sequence, axis=1)

# gff_data['encoded_seq'] = gff_data.apply(encode_sequence, axis=1) ; mirogen_data['encoded_seq'] = mirogen_data.apply(encode_sequence, axis=1)
gff_data['decoded_seq'] = gff_data['encoded_seq'].apply(decode_sequence) ; mirogen_data['decoded_seq'] = mirogen_data['encoded_seq'].apply(decode_sequence)

In [ ]:
print((gff_data[relevant_seq] == gff_data['decoded_seq']).value_counts())
# Iterate through the sampled false indices and check each point
# get sampled_false_indices (where gff_data['full_seq'] != gff_data['decoded_seq'])
false_indices = [idx for idx in gff_data[gff_data[relevant_seq] != gff_data['decoded_seq']].index]
sampled_false_indices = np.random.choice(false_indices, 3, replace=False)
for idx in sampled_false_indices:
    print("Index:", idx)

    # Extract necessary information from the DataFrame
    row = gff_data.loc[idx]
    full_seq = row[relevant_seq]
    # full_seq_folding = row['full_seq_folding'] ################################################
    encoded_seq = row['encoded_seq']
    decoded_seq = row['decoded_seq']

    # 1. Check Special Encoding Tokens
    # Check if special tokens are correctly replaced during decoding
    if 'ZZZZZ' in decoded_seq or 'BBBBB' in decoded_seq or 'DDDDD' in decoded_seq or 'FFFFF' in decoded_seq:
        print("Special Encoding Tokens not correctly replaced.")

    # 2. Check Start and End Indices
    # Print the start and end indices used during encoding
    mature_start = full_seq.find(row['Mature'])
    mature_end = mature_start + len(row['Mature'])
    star_start = row['Start_star']
    star_end = row['End_star']
    print("Mature Start:", mature_start, "Mature End:", mature_end)
    print("Star Start:", star_start, "Star End:", star_end)
    # Print the Decoded, Encoded, and Full Sequence for comparison
    print("Encoded Sequence:", encoded_seq)
    print("Decoded Sequence:", decoded_seq)
    print("full/hairpin Sequence:", full_seq)
    print('len(decoded_seq):', len(decoded_seq))
    print('len(full/hairpin seq):', len(full_seq))
    print("-" * 30)


True     23711
False      129
dtype: int64
Index: 22610
Mature Start: 50 Mature End: 72
Star Start: 1.0 Star End: 35.0
Encoded Sequence: atatcatataatatataatatatcaGACATDDDDDAAGGCAACGTGCCACTGATATGATAATTTATCATAFFFFFZZZZZTGATAATTTATCATATCAGTGGBBBBBCACGTTGCCTTATGTCTGATCGTTCGATGAGGGAAAAGCTTGG
Decoded Sequence: atatcatataatatataatatatcaGACATAAGGCAACGTGCCACTGATATGATAATTTATCATATGATAATTTATCATATCAGTGGCACGTTGCCTTATGTCTGATCGTTCGATGAGGGAAAAGCTTGG
full/hairpin Sequence: atatcatataatatataatatatcaGACATAAGGCAACGTGCCACTGATATGATAATTTATCATATCAGTGGCACGTTGCCTTATGTCTGATCGTTCGATGAGGGAAAAGCTTGG
len(decoded_seq): 130
len(full/hairpin seq): 115
------------------------------
Index: 13878
Mature Start: 42 Mature End: 64
Star Start: 1.0 Star End: 53.0
Encoded Sequence: AGTGCTGGAGCAAGAAGATCTCGTGCTGCGDDDDDACTCTACAAAGGAAAGTGCTTTCTGTTGCCAGAAAGAAGAGAAAGCGCTTCCCFFFFFZZZZZAAAGTGCTTTCTGTTGCCAGAABBBBBAGAAGAGAAAGCGCTTCCCTTTTGAGGGTTACGGTTTGAGAAAAGCAGTGTTGAAGTTG
Decoded Sequence: AGTGCTGGAGCAAGAAGATCTCGTGCTGCGACTCTACAAAGGAAAGT

In [ ]:
gff_sequences = ['<SOS>' + sequence + '<EOS>' for sequence in gff_data['encoded_seq']]
mirgendb_sequences = ['<SOS>' + sequence + '<EOS>' for sequence in mirogen_data['encoded_seq']]

In [ ]:
def calculate_statistics(sequences):
    num_sequences = len(sequences)
    avg_length = int(sum(len(seq) for seq in sequences) / num_sequences)
    min_length = min(len(seq) for seq in sequences)
    max_length = max(len(seq) for seq in sequences)

    return {
        "Number of sequences": num_sequences,
        "Average sequence length": avg_length,
        "Min sequence length": min_length,
        "Max sequence length": max_length

    }

In [ ]:
# pre mirna
print(calculate_statistics(gff_sequences))
print(calculate_statistics(mirgendb_sequences))

{'Number of sequences': 23840, 'Average sequence length': 150, 'Min sequence length': 135, 'Max sequence length': 202}
{'Number of sequences': 9747, 'Average sequence length': 150, 'Min sequence length': 139, 'Max sequence length': 164}


In [ ]:
def token_statistics(tokenizer, sequences):
    token_lengths = []

    for seq in sequences:
        tokens = tokenizer.encode(seq).tokens
        token_lengths.extend([len(token) for token in tokens])

    avg_length = np.mean(token_lengths)
    min_length = np.min(token_lengths)
    max_length = np.max(token_lengths)
    median_length = np.median(token_lengths)
    std_dev = np.std(token_lengths)

    return {
        "average": avg_length,
        "min": min_length,
        "max": max_length,
        "median": median_length,
        "std_dev": std_dev
    }



In [ ]:
def train_bpe_tokenizer(sequences, vocab_size=10000, min_frequency=3):
    special_tokens = ["<SOS>", "<EOS>", "ZZZZZ", "BBBBB", "DDDDD", "FFFFF"]

    # Define BPE model with an unknown token
    tokenizer = Tokenizer(BPE(unk_token="<UNK>"))

    # Define the BPE trainer with special tokens, vocab size, and min frequency
    trainer =  BpeTrainer(special_tokens=special_tokens, max_token_length=6) #vocab_size=vocab_size, min_frequency=min_frequency,
    # Train the tokenizer from the iterator of sequences
    tokenizer.train_from_iterator(sequences, trainer=trainer)

    # After training, add the special tokens to the tokenizer to ensure they won't be split
    tokenizer.add_tokens(special_tokens)

    return tokenizer

In [ ]:
bpe_tokenizer = train_bpe_tokenizer(gff_sequences + mirgendb_sequences, vocab_size=10000, min_frequency=3)
bpe_tokenizer.save("data/GPT_mature_star_bpe_tokenizer.json") # GPT_mature_star_bpe_hairpin_tokenizer / with flanks : "GPT_mature_star_bpe_tokenizer"

In [ ]:
sample_sequences = np.random.choice(gff_sequences,5,replace=False)
for seq in sample_sequences:
    # print(seq)
    encoded = bpe_tokenizer.encode(seq)
    print(f"Original: {seq}")
    print(f"Encoded: {encoded.tokens}\n")

Original: <SOS>DDDDDCCGGACGAACTTCCCAGCTCGGCCFFFFFGACATGAACGGGZZZZZTCGGTGGGATCTTCGTCCGTTTBBBBB<EOS>
Encoded: ['<SOS>', 'DDDDD', 'CCGGA', 'CGAA', 'CTTCC', 'CAGC', 'TCGG', 'CC', 'FFFFF', 'GA', 'CATG', 'AA', 'CGGG', 'ZZZZZ', 'TC', 'GGTG', 'GGATC', 'TTC', 'GTC', 'CGTTT', 'BBBBB', '<EOS>']

Original: <SOS>DDDDDAGGTTGGGATGTGTGCATTATTTGFFFFFTGGAAGGTTAGATCAZZZZZAATTGCACTCGTCCCGGCCTGCBBBBB<EOS>
Encoded: ['<SOS>', 'DDDDD', 'AGG', 'TTG', 'GGA', 'TGTG', 'TGCA', 'TTA', 'TTTG', 'FFFFF', 'TGGAA', 'GG', 'TTAGA', 'TCA', 'ZZZZZ', 'AA', 'TTGCA', 'CTC', 'GTC', 'CCGG', 'CCTGC', 'BBBBB', '<EOS>']

Original: <SOS>DDDDDAGTTCTTCACTGGCAAGCTTTAFFFFFTGTCCTCATGTACCAACTAZZZZZAAGCTGCCAGCTGAAGAACTGTBBBBB<EOS>
Encoded: ['<SOS>', 'DDDDD', 'AGTT', 'CTT', 'CACTG', 'GCAA', 'GC', 'TTTA', 'FFFFF', 'TGTC', 'CTCA', 'TGTA', 'CCAA', 'CTA', 'ZZZZZ', 'AA', 'GCTG', 'CCA', 'GCTG', 'AAGAA', 'CTGT', 'BBBBB', '<EOS>']

Original: <SOS>AZZZZZACCCGTAGATCCGAACTTGTGTBBBBBTGTCTGGCCACDDDDDACAAGCTCGTGTCTATAGGTATGFFFFF<EOS>
Encoded: ['<SOS>', '